## Tweeter 스크랩


## 필요한 라이브러리 설치
* 아나콘다 사용시 다음의 프롬프트 창을 열어 conda 명령어로 설치합니다.
* pip 사용시 아래에 있는 명령어를 터미널로 설치합니다.
<img src="https://i.imgur.com/Sar4gdw.jpg">

### Selenium
* `conda install -c anaconda selenium`
* [Selenium :: Anaconda Cloud](https://anaconda.org/anaconda/selenium)

* pip 사용시 : `pip install selenium`

### BeautifulSoup
* `conda install -c anaconda beautifulsoup4`
* [Beautifulsoup4 :: Anaconda Cloud](https://anaconda.org/anaconda/beautifulsoup4)

* pip 사용시 : `pip install beautifulsoup4`

### tqdm
* `conda install -c conda-forge tqdm`
* [tqdm/tqdm: A Fast, Extensible Progress Bar for Python and CLI](https://github.com/tqdm/tqdm)
* `pip install tqdm`

In [2]:
def get_login(driver, id, pwd):
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div/main/div/div/div/div[1]/div/a[2]/div/span/span').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[1]/label/div/div[2]/div/input').send_keys(id)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[2]/label/div/div[2]/div/input').send_keys(pwd)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[3]/div/div/span/span').click()
    time.sleep(3)

def get_postContents(driver, keyword, num_posts, interval):

    titles = []
    contents = []
    dates = []
    soup_titles = []
    soup_contents = []
    soup_dates = []
    post_df = pd.DataFrame(columns=('title', 'content', 'date'))
        
    # 지정된 회차 동안 반복하면서 스크롤을 화면 맨 아래로 이동한다.
    for i in trange(0, num_posts):
              
        # 현재 문서 높이를 가져와서 저장
        prev_height = driver.execute_script("return document.body.scrollHeight")
        
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 

        # 페이지 로딩 대기
        time.sleep(interval)  
            
        # 현재 화면 높이를 가져와서 저장한다
        curr_height = driver.execute_script("return document.body.scrollHeight")
        
        # 현재 브라우저에 표시되고 있는 소스코드 가져오기
        soup = bs(driver.page_source, 'html.parser')

        try:
#             print("필요한 정보만 추려낸다")
            soup = bs(driver.page_source, 'html.parser')
            soup_titles = soup.find_all('span', attrs={"class": "css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"})
            soup_contents = soup.find_all('span', attrs={"class": "css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"})
            soup_dates = soup.find_all('a', attrs={"class": "r-1re7ezh r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0 css-4rbku5 css-18t94o4 css-901oao"})

#             print("추려낸 내용을 깔끔하게 '개행문자와 공백 제거' 정리한다")
            for title, content, date in zip(soup_titles, soup_contents, soup_dates):
                title = title.get_text().replace("\n","").strip()
                titles.append(title)
                content = content.get_text().replace("\n","").strip()
                contents.append(content)
                date = date.get_text().replace("\n","").strip()
                dates.append(date)

        except:
            pass
        
#         if ( prev_height == curr_height ):
#             pass

#     print("추려낸 내용을 dataframe 에 저장 한다")
    post_df = pd.DataFrame({ "title": titles, "content": contents, "date": dates })
    
    return post_df.drop_duplicates()

def letsfind(keyword):
    
    num_posts = 2000
    interval = 2

    # 검색페이지로 이동한다
    url = f"https://twitter.com/search?q={keyword}%20until%3A2020-06-30%20since%3A2019-07-01&src=typed_query&f=live"
    driver.get(url)
    time.sleep(3)

    post_df = get_postContents(driver, keyword, num_posts, interval)
    # save to csv
    filename = "./scraps/" + "tweeter-scrapped_" + keyword.replace(" ","") + ".csv"   
    post_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

##############################################################################################
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd
from pandas import read_excel
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import datetime as dt
import time
from tqdm import trange
from tqdm import tqdm
##############################################################################################

# 로컬 PC
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# # Docker
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# options = webdriver.ChromeOptions()
# options.headless = True
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument("window-size=1920x1080") # 가상화면 크기
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# 크롬 드라이버 로드한다.
driver = webdriver.Chrome(options=options)
# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(5)
# 화면을 최대로 키운다
driver.maximize_window()

# 메인페이지로 이동한다
driver.get('https://twitter.com/')

# 로그인 한다
id = 'inhwan.jung@gmail.com'
pwd = '2020@alpha'
get_login(driver, id, pwd)
    
##################### 뉴스 검색 Main 루프 시작 ##################### 
# my_sheet = '소비키워드'
# file_name = 'deskresearch.xlsx'
# df = read_excel(file_name, sheet_name = my_sheet, header = 1) # index_col='번호'
# keywords = df['핵심단어'].values.tolist()

# 조회할 키워드를 설정한다.
keywords = ['외식', '식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']
keywords = ['비대면']

for keyword in keywords:
    letsfind(keyword)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [1:19:15<00:00,  2.38s/it]
